## SOM_BIs_comp_trans_yrstrain.ipynb

This notebooked identifies the optimum set of node clusters to define the SOM blocking index from the SOM dataset of best matching units (SOM_data) and the ground truth dataset (GTD)

-Does this analysis for getting the skill score for studying different training periods



In [1]:
90

90

In [2]:
import xarray as xr
import numpy as np
import matplotlib as mpl
mpl.use("agg", warn=False)
import matplotlib.pyplot as plt
import glob
import csv
import SOM_BIs_comp_funcs as SOMBI


## Apply cross-validation

In [3]:
#define variables
seas = "JJA" # season
var_str = "z500" # string to define variable in caption for histogram figure
skill_str = "precision" # skill metric to order the clusters by
persis_thresh = 5 # threshold for minimum number of days to identify blocking

GTD_str = "/rds/general/project/nowack_graven/live/carl_som_index/data/GTD_1979-2019_JJAextd.nc"
#GTD_str = "/rds/general/project/nowack_graven/live/carl_som_index/data/GTD_UKESM1-0-LL_piControl_1960-2060_JJAextd.nc"


row, col = 3, 3
k_idx = 2

if "UK" in GTD_str:
    rowcol = 3 # currently assuming square arrangement of nodes
    persis_thresh = 5
    #number of years in training dataset
    k_idx = 2
    k_arr =        [4, 5, 10,20]
    yrnum_cv_arr = [25,20,10, 5]
else:
    rowcol = 3 # currently assuming square arrangement of nodes
    k_idx = 4
    persis_thresh = 5
    #number of years in training dataset
    k_arr =        [ 2,  4, 5, 8, 10, 20]
    yrnum_cv_arr = [20, 10, 8, 5,  4,  2]
    
    
k = k_arr[k_idx]
yrnum_cv = yrnum_cv_arr[k_idx]

if "1979-2019" in GTD_str:
    GTD_sel = xr.open_dataset(GTD_str)['blocking']
    GTD = GTD_sel.sel(time=~((np.isin(GTD_sel.time.dt.month, [5, 9])) & (np.isin(GTD_sel.time.dt.day, [5, 6, 26, 27]))))
    mdl, ens = "era5", "reanal"
    var_str = "z"
    zg_str = "z"
    mdl_str = "era5"
    mdl_ens_str = "era5_reanal"
    swap_str = "_swapUKESM1-0-LL"    
    yrs_train = 37
if "UKESM" in GTD_str:
    GTD = xr.open_dataset("/rds/general/user/cmt3718/home/data/cmip6/GTD_UKESM1-0-LL/GTD_UKESM1-0-LL_piControl_1960-2060_JJAextd.nc")['blocking']#xr.open_dataset(GTD_str)['blocked']
    mdl, ens = "UKESM1-0-LL", "r1i1p1f2"
    var_str = "zg"
    zg_str = "zg"
    mdl_str = "UKESM1-0-LL_piControl"
    mdl_ens_str = "UKESM1-0-LL_piControl"
    swap_str = "_swapERA5"    
    yrs_train = 91
#only select JJA from GTD
F1_cv_cross_arr, F1_cv_F1_arr = [], []
yrst, yrend = int(GTD['time.year'].min()), int(GTD['time.year'].max())
yrst_cv_arr = [yrst+i*yrnum_cv for i in range(int((yrend - yrst)/yrnum_cv))]
domain, season = "EUR", "JJA_extd"

#use BMU_str if investigating the BMU+ effect
BMU_str=""#"_BMU+"

yrs_train_calc = True





In [4]:
zg_str, var_str = "ipv_330pt", "ipv_330pt"
#zg_str, var_str = "z", "z"
#zg_str, var_str = "msl", "msl"

In [ ]:


rowcols_arr = [(2, 1), (3, 1), (2, 2), (5, 1), (3, 2), (7, 1), (4, 2), (3, 3), (5, 2), (11, 1), (4, 3), (13, 1), (7, 2), (5, 3), (4, 4), (17, 1), (6, 3),
 (19, 1), (5, 4), (7, 3), (11, 2), (23, 1), (6, 4), (5, 5), (13, 2), (9, 3), (7, 4), (29, 1), (6, 5), (31, 1), (8, 4), (33, 1), (17, 2),
               (7, 5), (6, 6), (37, 1), (19, 2), (39, 1), (8, 5), (41, 1)]
notdone_nodenums = [17, 18, 19]

for row, col in rowcols_arr:
    if int(row)*int(col) in notdone_nodenums:
        yrs_train_str = f"{yrs_train}yrstrain" 
        print(f"yrs_train = {yrs_train}")
        years_cv_str_arr, F1_cv_cross_arr, pr_cv_cross_arr, rc_cv_cross_arr, F1_cv_F1_arr, pr_cv_F1_arr, rc_cv_F1_arr, clus_set_cross_arr, clus_set_F1_arr, len_clus_set_cross_arr, len_clus_set_F1_arr = ([], [], [], 
        [], [], [], [], [], [], [], [])

        for i, yrst_cv in enumerate(yrst_cv_arr):
            #if i == 0:
                #if yrst_cv == 1979:
                yrst_cv = yrst_cv+1
                print(f"rowcol = {row}x{col}")
                #define years used in cross-validation
                yrend_cv = yrst_cv+yrnum_cv-1
                print(yrst_cv, yrend_cv)
                #directories for the train and cv files
                dir_str = "/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/zg/"
                #f"/rds/general/user/cmt3718/home/data/reanal/SOM/trends/{var_str}/crossval/{k}-fold"
                #print(f"/rds/general/user/cmt3718/home/data/reanal/SOM/trends/{var_str}/crossval/{k}-fold")
    #/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/            
                if var_str == "msl" or mdl == "era5":
                    mdlstr="era5"
                    SOM_fstr_train = (f'/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/training_yrs/'
                    f'SOM_data_occur_train_{mdl}_{ens}_{row}x{col}_EUR_not{yrst_cv}-{yrend_cv}_JJA_extd_{var_str}_LTDManom{BMU_str}{yrs_train_str}.nc')
                    SOM_fstr_cv = (f'/rds/general/project/nowack_graven/live/carl_som_index/data/era5/{zg_str}/crossval/{k}-fold/training_yrs/'
                    f'SOM_data_occur_cv_{mdl}_{ens}_{row}x{col}_EUR_{yrst_cv}-{yrend_cv}_JJA_extd_{var_str}_LTDManom{BMU_str}{yrs_train_str}.nc')
                else:
                    mdlstr="UKESM1-0-LL_piControl"
                    SOM_fstr_train = (f'/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/training_yrs/'
                    f'SOM_data_occur_train_{mdl}_{ens}_{row}x{col}_EUR_not{yrst_cv}-{yrend_cv}_JJA_extd_{var_str}_LTDManom{BMU_str}{yrs_train_str}.nc')
                    SOM_fstr_cv = (f'/rds/general/project/nowack_graven/live/carl_som_index/data/UKESM1-0-LL_piControl/{zg_str}/crossval/{k}-fold/training_yrs/'
                    f'SOM_data_occur_cv_{mdl}_{ens}_{row}x{col}_EUR_{yrst_cv}-{yrend_cv}_JJA_extd_{var_str}_LTDManom{BMU_str}{yrs_train_str}.nc')

                cluster_set_plot_train = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/{mdlstr}/{var_str}/crossval/{k}-fold/training_yrs/plots/"
                            f"z_F1_{k}-fold_{mdl}_{ens}_{row}x{col}_EUR_not{yrst_cv}-{yrend_cv}_JJA_extd_{var_str}_LTDManom_train{BMU_str}{yrs_train_str}.png")
                cluster_set_plot_train_hist = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/{mdlstr}/{var_str}/crossval/{k}-fold/training_yrs/plots/"
                            f"z_F1_{k}-fold_{mdl}_{ens}_{row}x{col}_EUR_not{yrst_cv}-{yrend_cv}_JJA_extd_{var_str}_LTDManom_train_hist{BMU_str}{yrs_train_str}.png")
                cluster_set_plot_cv = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/{mdlstr}/{var_str}/crossval/{k}-fold/training_yrs/plots/"
                            f"z_F1_{k}-fold_{mdl}_{ens}_{row}x{col}_EUR_not{yrst_cv}-{yrend_cv}_JJA_extd_{var_str}_LTDManom_cv{BMU_str}{yrs_train_str}.png")
                cluster_set_plot_cv_hist = (f"/rds/general/project/nowack_graven/live/carl_som_index/data/{mdlstr}/{var_str}/crossval/{k}-fold/training_yrs/plots/"
                            f"z_F1_{k}-fold_{mdl}_{ens}_{row}x{col}_EUR_not{yrst_cv}-{yrend_cv}_JJA_extd_{var_str}_LTDManom_cv_hist{BMU_str}{yrs_train_str}.png")            

                #dayofyr_xr=[GTD.sel(time=(slice(f"{yr}-05-28", f"{yr}-09-04")))['time'] for yr in range(int(yrst),int(yrend)+1)]    
                #select DJF to test TM90 index behaviour
                #concatenate the lists of boolean DataArray objects - also takes a few seconds
                #dayofyr_xr_concat = xr.concat(dayofyr_xr, "time")
                #GTD_sel = GTD.sel(time = dayofyr_xr_concat['time']) #remove excess days not in JJA extd
                
                SOM_data_train = xr.open_dataset(SOM_fstr_train)['SOM_data'] #the results from the training of the SOM on the training dataset
                SOM_data_cv = xr.open_dataset(SOM_fstr_cv)['SOM_data']
                GTD_train = GTD.sel(time=np.isin(GTD['time'], SOM_data_train['time']))
                GTD_cv = GTD.sel(time=np.isin(GTD['time'], SOM_data_cv['time'], invert=False))                   
                GTD_train = GTD.sel(time = np.isin(GTD['time.year'], np.arange(yrst_cv, yrend_cv+1), invert=True))
                GTD_cv = GTD.sel(time = np.isin(GTD['time.year'], np.arange(yrst_cv, yrend_cv+1), invert=False))                 
             
                #print(f"SOM_data_train = {SOM_data_train}")
                #print(f"GTD_train = {GTD_train}")
                #print(f"SOM_data_cv = {SOM_data_cv}")                
                #print(f"GTD_cv = {GTD_cv}")
                nodes_arr = SOMBI.create_nodes_arr(row, col)
                #print("calc best cluster set")
                #best_cluster_set_cross is the cluster set when precision and recall cross
                #best_cluster_set_F1 is the cluster set for the best F1 score
                #print(f"best_cluster_set_F1 = {best_cluster_set_F1}")
                #print(f"len(best_cluster_set_F1) = {len(best_cluster_set_F1)}")
                SOM_clusters_block, blocked_days, persis_thresh, SOM_nodes, best_cluster_set_cross, best_cluster_set_F1, optimal_prec, optimal_recall, optimal_F1 = SOMBI.plot_SOM_BI_performance(
                    row, col, SOM_data_train, skill_str, cluster_set_plot_train, cluster_set_plot_train_hist, GTD_train, var_str, seas, persis_thresh)
                #calculate the F1_cv score for two different methods of choosing the F1 scores
                F1_cv_cross, pr_cv_cross, rc_cv_cross = SOMBI.calc_F1_cv(SOM_data_cv, GTD_cv, best_cluster_set_cross, persis_thresh, nodes_arr, seas)
                #print(f"calc F1 cv for crossing-point {F1_cv_cross}")            
                F1_cv_F1, pr_cv_F1, rc_cv_F1 = SOMBI.calc_F1_cv(SOM_data_cv, GTD_cv, best_cluster_set_F1, persis_thresh, nodes_arr, seas)
                #print(f"calc F1 cv for F1 {F1_cv_F1}")
                years_cv_str = f"{yrst_cv}-{yrend_cv}"
                years_cv_str_arr.append(years_cv_str)
                F1_cv_cross_arr.append(F1_cv_cross)
                pr_cv_cross_arr.append(pr_cv_cross)
                rc_cv_cross_arr.append(rc_cv_cross)
                clus_set_cross_arr.append(best_cluster_set_cross)
                len_clus_set_cross_arr.append(len(best_cluster_set_cross))

                F1_cv_F1_arr.append(F1_cv_F1)
                print(f"F1_cv_cross, pr_cv_cross, rc_cv_cross = {F1_cv_cross}, {pr_cv_cross}, {rc_cv_cross}")
                print(f"len(best_cluster_set_F1) = {len(best_cluster_set_F1)}")                
                print(f"F1_cv_F1, pr_cv_F1, rc_cv_F1 = {F1_cv_F1}, {pr_cv_F1}, {rc_cv_F1}")
                print(f"len(best_cluster_set_F1) = {len(best_cluster_set_F1)}")
                pr_cv_F1_arr.append(pr_cv_F1)
                rc_cv_F1_arr.append(rc_cv_F1)
                clus_set_F1_arr.append(best_cluster_set_F1)
                len_clus_set_F1_arr.append(len(best_cluster_set_F1))

        print(f"GEN SKILL for {row}x{col} {k}-fold: \n F1 crossing point = {np.mean(F1_cv_cross_arr)} \n F1 arr = {np.mean(F1_cv_F1_arr)}")
        #write the cross-validation F1 scores to an xarray Dataset
        F1_cv_ds = xr.Dataset({'years_cv': years_cv_str_arr, 'F1_cv_cross': F1_cv_cross_arr,'pr_cv_cross': pr_cv_cross_arr, 'rc_cv_cross': rc_cv_cross_arr,
                       'num_clus_cross': str(xr.concat(clus_set_cross_arr, dim = "clusters").values),
                        'F1_cv_F1': F1_cv_F1_arr, 'pr_cv_F1': pr_cv_F1_arr, 'rc_cv_F1': rc_cv_F1_arr, 
                               'clus_set_F1': str(xr.concat(clus_set_F1_arr, dim = "clusters").values), 'num_clus_F1': len_clus_set_F1_arr})

        SOM_cv_F1_str = (f'/rds/general/project/nowack_graven/live/carl_som_index/data/{var_str}/crossval/F1_cv/{k}-fold/training_yrs/'
        f'cv_crossvsF1_{k}-fold_{mdl}_{ens}_{row}x{col}_EUR_{yrnum_cv}yrs_JJA_extd_{var_str}_LTDManom_pr_rc{BMU_str}{yrs_train_str}.nc')
        F1_cv_ds.to_netcdf(SOM_cv_F1_str)
        print(f'saved /rds/general/project/nowack_graven/live/carl_som_index/data/{var_str}/crossval/F1_cv/{k}-fold/training_yrs/'
        f'cv_crossvsF1_{k}-fold_{mdl}_{ens}_{row}x{col}_EUR_{yrnum_cv}yrs_JJA_extd_{var_str}_LTDManom_pr_rc{BMU_str}{yrs_train_str}.nc')


yrs_train = 37
rowcol = 17x1
1980 1983
Calculating skill of individual clusters
Calculating skill of combined clusters
peak F1 score = 0.705592105263158
prec for peak F1 score = 0.6534653465346535
recall for peak F1 score = 0.7667560321715817
number of clusters for peak F1 score = 196
F1_cv_cross, pr_cv_cross, rc_cv_cross = 0.4067796610169491, 0.4948453608247423, 0.34532374100719426
len(best_cluster_set_F1) = 196
F1_cv_F1, pr_cv_F1, rc_cv_F1 = 0.39834024896265563, 0.47058823529411764, 0.34532374100719426
len(best_cluster_set_F1) = 196
rowcol = 17x1
1984 1987
Calculating skill of individual clusters
Calculating skill of combined clusters
peak F1 score = 0.7285768214420536
prec for peak F1 score = 0.6489576328177539
recall for peak F1 score = 0.8304647160068847
number of clusters for peak F1 score = 223
F1_cv_cross, pr_cv_cross, rc_cv_cross = 0.29189189189189185, 0.30337078651685395, 0.28125
len(best_cluster_set_F1) = 223
F1_cv_F1, pr_cv_F1, rc_cv_F1 = 0.29245283018867924, 0.267241379310